# Szakdolgozat 
### Véges állapotú automaták tanulása genetikus algoritmussal
#### Szerző: Csikós Dominik
#### Témavezető: Borbély Gábor

In [ ]:
import random
import numpy as np
import os
import shutil
import subprocess
import math
import operator
from graphviz import Digraph
from graphviz import Source
import sys

### Hiperparaméterek:

In [ ]:
start_pop = 'gens/gen'
pop_size = 20
max_it = 200
wfsa_path = 'wfsa_win64/wfsa.exe'
data_path = 'wfsa/data/proquants.txt'
delta = 0


### Automata beolvasása egy dictionarybe függvény:

In [ ]:
def readautomaton(filename):
    with open(filename) as f:
        content = f.readlines()
    A = dict()
    for i in range(3,len(content)):
        sor = []
        l = content[i].split(" ")
        a = l[0]
        for j in range(1,len(l),2):
            sor.append(l[j])
        try:
            sor.remove('\n')
        except:
            pass
        if a in A:
            A[a].append(sor)
        else:
            A[a] = [sor]
    return A

### Két állapot összeillesztésére függvény:

In [ ]:
def rmdups(x):
    return list(dict.fromkeys(x))

In [ ]:
def mergestate(A,s,t):
    l = len(A)
    l1 = l+1
    name = 'HUB' + str(l1) #új név létrehozás
    while name in A:
        name = name + '.'
    for i in A: #új állapot bemeneteinek felírása
        if len(A[i][1]) != 0:
            if (s in A[i][1]):
                A[i][1].remove(s)
                A[i][1].append(name)
            if (t in A[i][1]):
                A[i][1].remove(t)
                A[i][1].append(name)
        A[i][1] = rmdups(A[i][1])
    A[str(name)] = [list(set(A[s][0]) | set(A[t][0]))] #új állapot emissziói
    out = list(set(A[s][1]) | set(A[t][1])) #új állapot kimenetei(önmaga is akár)
    A[str(name)].append(out) #új állapot kimenetei
    del A[s]
    del A[t]
    return A

### Egy állapot szétszedésére függvény:

In [ ]:
def splitstate(A,s):
    v = []
    r = []
    #s-hez kitalálni új nevet: s_uj, figyelni h uj nev legyen
    l = ''.join([i for i in s if not i.isdigit()])
    j = 1
    while (l+(str(j))) in A:
        j += 1
    sn = l+str(j)
    for i in range(len(A[str(s)][0])): #elemek kiválasztása, melyek átkerülnek az új állapotba
        if random.random() > 0.5:
            v.append(A[s][0][i])
        else:
            r.append(A[s][0][i]) #ez lesz a regi
    A[sn] = [v] #új állapot létrehozás
    A[sn].append(A[s][1])#új állapot kimenetek(ua, mint a régi állapot kimenetei)
    for i in range(len(v)): #az új állapotban lévő elemek kiszedése a régi állapotból
        A[s][0].remove(v[i])
    for x in A:
        if len(A[x][1])!=0:
            if s in A[x][1]:
                #x->s átmenet létezik
                A[x][1].append(sn)
            A[x][1] = rmdups(A[x][1])
    return A

### Automatát fájlba kiíró fv:

In [ ]:
def writeautomaton(A,filename):
    with open (filename,'w',newline='\n') as f:
        print('', file=f )
        print('^', file=f)
        print('$', file=f )
        for x in A:
            if len(A[x][0]) == 0:
                raise ValueError(str(x)+' has no emission!')
            else:
                print(x, file=f, end='')
                for i in A[x][0]:
                    print(' ' + i + ' 0',file=f, end='')
                print('',file=f)
            print(x, file=f, end='')
            if (A[x][1] != 0):
                for j in A[x][1]:
                    print(' ' + j + ' 0',file=f, end='')
            else:
                raise ValueError(str(x)+' has no transition!')
            print('',file=f)


### Függvény, ami beolvassa az adott mappából az összes automata nevét:

In [ ]:
def readfolderwfsa(foldername): #mappabol beolvassa a wfsa fajlok neveit
    A = []
    g = os.listdir(foldername)
    for i in g:
        l = i.split(".")
        if l[-1] == 'wfsa':
            A.append(i)
        else: 
            continue
    return A

### Függvény, ami beolvassa az adott mappából az összes kiértékelt fájl nevét:


In [ ]:
def readfolderevaled(foldername):
    A = []
    g = os.listdir(foldername)
    for i in g:
        l = i.split(".")
        if l[-1] == 'evaled':
            A.append(i)
        else: 
            continue
    return A

In [ ]:
def learnedname(filename): #hozzateszi a filenevhez h .learned
    f = filename + '.learned'
    return f

In [ ]:
def evaledname(filename):
    f = filename + '.evaled'
    return f

### Kiértékelő függvény:

In [ ]:
def evaluation(filename,nn=10**9):
    with open(learnedname(filename), 'w') as f:
        result = subprocess.run([wfsa_path,
                        '-c', data_path, 
                        '-a', filename, '-n', '-eval','-i','8'], 
                        encoding='utf8', stderr=subprocess.PIPE, stdout=f)
    if result.returncode == 0:
        r = result.stderr.split('Result: ')[-1].strip().split(' ')
        n = []
        #print(result.stderr) #kiírjuk-e a részleteket is
        try:
            for i in r:
                n.append(float(i))
        except:
            return(float('inf'))
        e = max(n[0]+n[1] - delta,0)+((n[2]+n[3])/nn)+((n[4]+n[5])/(2*nn))+(n[6]/(2*nn))*math.log(nn/(2*math.pi))
        return e
    else:
        return(float('inf'))

### Egész mappát kiértékelő függvény, majd egy gyors kiértékelő függvény:

In [ ]:
def evaluatefolder(foldername): #egesz mappat kiértékel
    A = readfolderwfsa(foldername)
    for i in A:
        if os.path.isfile(foldername + str('/') + evaledname(i)):
            continue
        else:
            with open(evaledname(foldername + str('/') + i),'w+') as f:
                print(evaluation(foldername + str('/') + i), file=f)

In [ ]:
def evaluatefast(filename):
    if os.path.isfile(evaledname(filename)):
        with open (evaledname(filename)) as f:
            content = f.readlines()
        return float(content[0])
    else:
        with open(evaledname(filename),'w+') as f:
            print(evaluation(filename), file=f)
        return evaluation(filename)

### Létrehoz egy mappát, vagy ha már létezik ez a mappa, törli majd újra létrehozza:

In [ ]:
def safenewdirectory(foldername):
    if os.path.isdir(foldername):
        shutil.rmtree(foldername)
        os.mkdir(foldername)
    else:
        os.mkdir(foldername)

### Mutációfüggvény

In [ ]:
def mutate(V):
    r = random.random()
    if r > 0.5:
        if len(list(V.keys())) < 3: 
            return(V)
        else:
            l = random.sample(list(V.keys()),2)
            s = l[0]
            t = l[1]
            if (s == '^') or (t == '^'): #ne mergelje ^t
                return(V)
            else:
                try:
                    new = mergestate(V,s,t)
                    return(new)
                except:
                    return(V)
    else:
        s = random.choice(list(V.keys()))
        if len(V[str(s)][0])<2: #ne bontson szet 5nél kisebb allapotot
            return(V)
        else:
            if s == '^': #ne splitelje ^t
                return(V)
            else:
                try:
                    new = splitstate(V,s)
                    return(new)
                except:
                    return(V)

In [ ]:
def rmnumbers(filename):
    result = ''.join([i for i in filename if not i.isdigit()])
    l = result.split('.')
    return l

### Kiválasztó függvény, a pop_size változó szerint tartja meg a legjobbakat:

In [ ]:
def selection(foldername):
    evaluatefolder(foldername)
    E = readfolderwfsa(foldername)
    l = dict()
    for i in range(0,len(E)):
        l[E[i]] = evaluatefast(foldername + '/' + E[i])
    sorted_l = sorted(l.items(), key=operator.itemgetter(1))
    h = len(sorted_l)
    for i in range(0,len(sorted_l)):
        if i < pop_size: #ezzel lehet varialni
            continue
        else:
            os.remove(foldername + '/' + sorted_l[i][0])
            os.remove(learnedname(foldername + '/' + sorted_l[i][0]))
            os.remove(evaledname(foldername + '/' + sorted_l[i][0]))
    print(sorted_l[0])
    print(foldername)

### Új generáció létrehozása az előzőből(pg:previous generation):

In [ ]:
def createnewgen2(pg):
    files = os.listdir(start_pop + str(pg))
    safenewdirectory(start_pop + str(pg+1))
    for f in files:
        shutil.copy(start_pop + str(pg) +'/' + f, start_pop + str(pg+1))
    A = readfolderwfsa(start_pop + str(pg+1))
    for a in A:
        V = readautomaton(start_pop + str(pg+1) + '/' + a)
        new = mutate(V)
        i = 0
        l = rmnumbers(a)
        while os.path.exists(start_pop + str(pg+1) + '/' + l[0] + str(i) + "." + l[1]):
            i += 1
        try:
            writeautomaton(new,start_pop + str(pg+1) + '/' + l[0] + str(i) + "." + l[1])
        except:
            continue
            #writeautomaton(V,start_pop + str(pg+1) + '/' + l[0] + str(i) + "." + l[1])
    selection(start_pop + str(pg+1))

### A genetikus algoritmus futtatása:

In [ ]:
%%time
for i in range(1,max_it):
    createnewgen2(i)

### Automata kirajzolása graphwiz segítségével:

In [ ]:
def readautomatondraw(filename):
    with open(filename) as f:
        content = f.readlines()
    A = dict()
    for i in range(3,len(content)):
        sor = []
        l = content[i].split(" ")
        a = l[0]
        s = '\\'
        for j in range(1,len(l)):
            if s in r"%r" % l[j]:
                l[j] = l[j][:-1]
            sor.append(l[j])
        if a in A:
            A[a].append(sor)
        else:
            A[a] = [sor]
    return A

In [ ]:
def gvfilename(filename):
    filename = filename +'.gv'
    return filename

#### Csak az átmenetek ábrázolása:

In [ ]:
def togvout(filename): #outs to graph
    A = readautomatondraw(filename)
    with open (gvfilename(filename),'w') as f:
        print('digraph wfsa {', file=f )
        print('\trankdir = LR;', file=f)
        print('\tsize = "8.5"', file=f)
        print('\tnode [shape = circle];', file=f)
        for i in A:
            for j in range(0,len(A[i][1]),2):
                print('\t"'+str(i) + '" -> "' + str(A[i][1][j]) + '" [ label = "' + str(A[i][1][j+1]) + '"];', file = f)
            #for j in range(0,len(A[i][1]),2):
            #    print('\t"'+str(i) + '" -> "' + str(A[i][1][j]) +'";', file = f)
            #for j in range(0,len(A[i][0]),2):
            #    print('\t"'+str(i) + '" -> "' + str(A[i][0][j]) + '" [ label = "' + str(A[i][0][j+1]) + '"];', file = f)
        print('}', file = f, end='')

#### Csak a kimenetek ábrázolása:

In [ ]:
def togvem(filename): #emissions to graph
    A = readautomatondraw(filename)
    with open (gvfilename(filename),'w') as f:
        print('digraph wfsa {', file=f )
        print('\trankdir = LR;', file=f)
        print('\tsize = "8.5"', file=f)
        print('\tnode [shape = circle];', file=f, end='')
        for i in A:
            print(' "' + str(i) + '"', file= f, end = '')
            print('"$"', file = f, end = '')
        print(';',file= f)
        print('\tnode [shape = box];', file=f)
        for i in A:
            for j in range(0,len(A[i][1]),2):
                print('\t"'+str(i) + '" -> "' + str(A[i][1][j]) + '" [ label = "' + str(A[i][1][j+1]) + '"];', file = f)
            #for j in range(0,len(A[i][0]),2):
            #    print('\t"'+str(i) + '" -> "' + str(A[i][0][j]) + '" [ label = "' + str(A[i][0][j+1]) + '"];', file = f)
        print('}', file = f, end='')

In [ ]:
def createdrawem(filename):
    togvem(filename)
    A = Source.from_file(gvfilename(filename))
    A.view()

In [ ]:
def createdrawout(filename):
    togvout(filename)
    A = Source.from_file(gvfilename(filename))
    A.view()

### LaTeX forráskód létrehozása, mely táblázatos formában ábrázolja az automatát:

In [ ]:
def specialchars(szoveg):
    if szoveg=='$':
        return('\\$')
    if szoveg=='^':
        return('$\\wedge$')
    if szoveg=='':
        return('<üres sztring>')
    else:
        return szoveg

In [ ]:
def totable(filename):
    A = readautomaton(filename)
    print('\\begin{table}')
    print('\\begin{tabular}{|l|p{6cm}|p{5cm}|}\n\hline \nÁllapot & Emissziók & Kimenetek\\\\\n\\hline\n\\hline')
    for a in A:
        u = specialchars(a)
        print(u+' & ',end='')
        for i in range(0,len(A[a][0]),1):
            v = specialchars(A[a][0][i])
            if i == len(A[a][0])-1:
                print(v, end = '') 
            else:
                print(v + ', ',end='')
        print(' & ',end='')
        for i in range(0,len(A[a][1]),1):
            w = specialchars(A[a][1][i])
            if i == len(A[a][1])-1:
                print(w, end = '') 
            else:
                print(w + ', ',end='')
        print('\\\\')
        print('\\hline')
    print('\\end{tabular}')
    print('\\caption{}')
    print('\\end{table}')